In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pylab as pl

import os
import sys

import numpy as np
from scipy import ndimage as ni
import maxflow

from tifffile import imread, imsave
import cPickle as pickle

from skimage.filters import gaussian_filter

from spimagine import volshow

%reload_ext autoreload
%autoreload 1

%matplotlib qt

# %matplotlib inline
# %pylab inline
# pylab.rcParams['figure.figsize'] = (5, 5)

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


### Load sampled unit sphere and set parameters

In [2]:
plt.rcParams['figure.figsize'] = (5, 5)

INF = 10000000
filename = "sample3d_cup1.tif"

# load pickeled unit sphere sampling
with open('sphere_sampling.pkl','r') as f:
    dictSphereData = pickle.load(f)
    
# sampling parameters
num_columns = dictSphereData['n']
col_vectors = dictSphereData['points']
neighbors = dictSphereData['neighbors']
num_neighbors = len(neighbors)/num_columns
K = 10       # as in Wu & Chen 2002

# net surface related parameters
max_delta_k = 4

# positioning of net in image
center_x = 130
center_y = 160
center_z = 12
max_r_x = 40
max_r_y = 40
max_r_z = 12

In [3]:
neighbors_of = np.ones([num_columns,num_neighbors]) * -1
for i,p1 in enumerate(col_vectors):
    dists = []
    for j,p2 in enumerate(col_vectors):
        dists.append( [i, j, np.dot(p1,p2)] )
    sorted_dists = sorted(dists, key=lambda dists: -dists[2])
    for idx in range(1,1+num_neighbors):
        neighbors.append(sorted_dists[idx])
    for idx in range(num_neighbors):
        neighbors_of[i,idx] = sorted_dists[idx+1][1]

print neighbors_of

[[   3.    2.    5.    1.    8.    6.]
 [   6.    4.    0.    9.    3.    2.]
 [   7.    0.   10.    5.   15.    4.]
 ..., 
 [ 504.  511.  501.  506.  496.  507.]
 [ 505.  507.  511.  502.  508.  509.]
 [ 508.  509.  506.  510.  503.  505.]]


### Loading image to be segmented

In [4]:
sigma = 2.
image = imread(filename)
image_smooth = gaussian_filter(image,sigma)
print 'Image dimensions: ', image.shape
print 'Image min/max:    ', np.min(image), np.max(image)

Image dimensions:  (25, 335, 275)
Image min/max:     199 481


In [5]:
volwin = volshow(image, stackUnits = [1.,1.,4.], raise_window=False)
# volwin = volshow(image_smooth, stackUnits = [1.,1.,4.], raise_window=False)

**Loading membrane probs computed with ilastik** (just testing...;)

In [6]:
image_memprob = np.load("sample3d_cup1_Probabilities.npy")[:,:,:,0]
volwin = volshow(image_memprob, stackUnits = [1.,1.,4.], raise_window=False)

# Building Graph

### Function definitions

In [7]:
import bresenham as bham
from myfunctions import *

In [8]:
bham.bresenhamline(np.array([[9,6,3]]), np.array([[0,0,0]]))

array([[8, 5, 3],
       [7, 5, 2],
       [6, 4, 2],
       [5, 3, 2],
       [4, 3, 1],
       [3, 2, 1],
       [2, 1, 1],
       [1, 1, 0],
       [0, 0, 0]])

### Compute weights *w*

In [12]:
w = np.zeros([num_columns, K]) # node weights
w_tilde = np.zeros([num_columns, K])

# fill in node weights
for i in range(num_columns):
    to_x = int(center_x + col_vectors[i,0]*max_r_x)
    to_y = int(center_y + col_vectors[i,1]*max_r_y)
    to_z = int(center_z + col_vectors[i,2]*max_r_z)
#     print 'from (', center_x, center_y, center_z, ') to (', to_x, to_y, to_z, ')'
    coords = bham.bresenhamline(np.array([[center_x, center_y, center_z]]), np.array([[to_x, to_y, to_z]]))
    num_pixels = len(coords)
    for k in range(K):
        start = int(k * float(num_pixels)/K)
        end = max( start+1, start + num_pixels/K )
        w[i,k] = -1 * compute_weight( image, coords[start:end])
#         print '   coords:        ', coords[start:end]
#         print '   indizes and w: ', start, end, w[i,k]
#     break
        
for i in range(num_columns):
    w_tilde[i,0] = w[i,0] 
    for k in range(1,K):
        w_tilde[i,k] = w[i,k]-w[i,k-1]

print w, np.min(w), np.max(w)
print w_tilde

[[-269. -259. -270. ..., -245. -250. -253.]
 [-269. -269. -260. ..., -252. -251. -251.]
 [-273. -265. -267. ..., -252. -249. -254.]
 ..., 
 [-254. -262. -246. ..., -260. -257. -271.]
 [-254. -251. -250. ..., -256. -252. -259.]
 [-254. -247. -241. ..., -259. -256. -278.]] -411.0 -234.0
[[-269.   10.  -11. ...,   -1.   -5.   -3.]
 [-269.    0.    9. ...,   -5.    1.    0.]
 [-273.    8.   -2. ...,    0.    3.   -5.]
 ..., 
 [-254.   -8.   16. ...,   -6.    3.  -14.]
 [-254.    3.    1. ...,   -2.    4.   -7.]
 [-254.    7.    6. ...,   -1.    3.  -22.]]


### Build flow network

In [13]:
num_nodes = num_columns*K
num_edges = (num_nodes*num_neighbors*(max_delta_k+max_delta_k+1))/2

g = maxflow.Graph[float](num_nodes,num_edges)
nodes = g.add_nodes(num_nodes)

for i in range(num_columns):

    # connect column to s,t
    for k in range(K):
        if w_tilde[i,k] < 0:
            g.add_tedge(i*K+k, -w_tilde[i,k], 0)
        else:
            g.add_tedge(i*K+k, 0, w_tilde[i,k])
            
    # connect column to i-chain
    for k in range(1,K):
        g.add_edge(i*K+k, i*K+k-1, INF, 0)
        
    # connect column to neighbors
    for k in range(K):
        for j in neighbors_of[i]:
            k2 = max(0,k-max_delta_k)
            g.add_edge(i*K+k, j*K+k2, INF, 0)
            # print i,k,int(j),k2

### Solve and show

In [14]:
maxval = g.maxflow()

size_s_comp = 0
size_t_comp = 0
for n in nodes:
    seg = g.get_segment(n)
    if seg == 0:
        size_s_comp += 1
    else:
        size_t_comp += 1
    #print "Segment of the node ",n,":", seg
print "Size of s and t component: ", size_s_comp, size_t_comp

Size of s and t component:  3230 1890


In [28]:
imgout = np.zeros(image.shape)
for i in range(num_columns):
    for k in range(K):
        x = int(center_x + col_vectors[i,0] * k/float(K) * max_r_x)
        y = int(center_y + col_vectors[i,1] * k/float(K) * max_r_y)
        z = int(center_z + col_vectors[i,2] * k/float(K) * max_r_z)
        seg = g.get_segment(i*K+k)
        if seg == 0:
            if x>=0 and y>=0 and z>=0 and \
               x<image.shape[2] and y<image.shape[1] and z<image.shape[0]:
                imgout[z,y,x] = 1
# IDEE: draw line to all neighbors (at height k) along 3d bresenham

In [29]:
volwin = volshow(imgout, stackUnits = [1.,1.,4.], raise_window=True)